# Practicum OO - AOP
Dit bestand bevat de practicumopgaven voor de module OO&AOP van het vak Advanced Technical Programming (TCIT-VKATP-17). 

Een Jupyter Notebook bestaat uit cells met code of toelichting. Sommige code-cells hoeven enkel uitgevoerd te worden om functies beschikbaar te maken, anderen moet je zelf in vullen (opdrachten staan duidelijk met kopjes gemarkeerd, je moet dan zelf de delen met TODO invullen). Je kunt de code per cel uitvoeren door de cel te selecteren en "run cell" te kiezen. De cel direct hieronder laadt de nodige modules in. Als je code niet naar verwachting werkt kan het zijn dat je deze cell moet runnen, of een van de andere cels vóór de cel met error. Let erop dat als je de Jupyter-server afsluit en later verder gaat moet je deze cells opnieuw runnen.

In [10]:
from unittest import *    
import doctest
import io
import inspect
from contextlib import redirect_stdout

__Opdrachten die zijn gemarkeerd met _[PORTFOLIO]_ dienen te worden opgenomen in je portfolio voor de eindopgave__.

Laat deze opdrachten door je docent aftekenen aan het einde van de les (of aan het begin van de volgende les).

***
## College 2: Reflectie en metaprogrammeren
Deze opdrachten gaan over het college over reflectie en metaprogrammeren en dienen aan het einde van dat college te worden gemaakt.

Maak ook de opdrachten in de reader!


#### Opdracht 2.1 — OO in Python [Portfolio]
Een Queue is gebaseerd op het FIFO-principe: First In First Out

<img src="queue.png" width="500px">

`enqueue` voegt een element achteraan toe.  
`dequeue` haalt een element vooraan weg.

De volgende klasse implementeert Queue:

In [8]:
class MyQueue(list):
    def __init__(self, a=[]):
        list.__init__(self, a)
    
    def dequeue(self):
        return self.pop(0) # geen fout-afhandeling
    
    def enqueue(self, x):
        self.append(x)

Python heeft een aantal ingebouwde queue-klassen: Queue, LifeQueue, PriorityQueue, en deque.

`deque` staat voor double-ended queue: aan beide kanten van de queue kan worden toegevoegd en verwijderd.

Breidt de klasse `MyQueue` uit tot de klasse `MyDeQue`.

De klasse heeft de volgende methoden:
* `appendright(x)`: voeg `x` aan de rechterkant toe
* `appendleft(x)`: voeg `x` aan de linkerkant toe
* `popright()`: verwijder en retourneer het element dat helemaal rechts staat
* `popleft()`: verwijder en retourneer het element dat helemaal links staat
* `reverse()`: keer de elementen in de queue om
* `rotateright(n)`: verschuif de elementen `n` posities naar rechts.
  Als `n < 0`: verschuif de elementen (`-n`)  posities naar links.
* `rotateleft(n)`: verschuif de elementen `n` posities naar links.
  Als `n < 0`: verschuif de elementen (`-n`) posities naar rechts.
  
Maak zoveel mogelijk gebruik van de andere methoden en methoden uit de super-klassen. Zorg voor een goede fout-afhandeling.

Denk er ook aan dat je voldoende tests schrijft om de functionaliteit zoals hierboven gegeven kan garanderen!

#### Opdracht 2.2 — `eval` en veiligheid [Portfolio]

Raadpleeg de site https://www.programiz.com/python-programming/methods/built-in/eval

Een eenvoudige interactieve expressie-evaluator is de volgende:

In [ ]:
while True:
    r = input("Type an expression: ")
    print("Result:", eval(r))

Type an expression: 4+4
Result: 8


In [11]:
class MyDeQue(MyQueue):
    def __init__(self, a=[]):
        MyQueue.__init__(self, a)

    def appendright(self, x):
        self.enqueue(x)

    def appendleft(self, x):
        self.insert(0, x)

    def popright(self):
        if len(self):
            return self.pop(len(self) - 1)

    def popleft(self):
        return self.dequeue()

    def reverse(self):
        super().reverse()

    def rotateright(self, n):
        for _i in range(n):
            self.appendleft(self.popright())

    def rotateleft(self, n):
        for _i in range(n):
            self.appendright(self.popleft())


class MyDeQueTest(TestCase):
    def setUp(self):
        self.testList = [4, 2, 3, 6, 8, 9]
        self.deque = MyDeQue(self.testList)

    def testPopLeft(self):
        self.assertEqual(4, self.deque.popleft())

    def testPopRight(self):
        self.assertEqual(9, self.deque.popright())

    def testAppendRight(self):
        self.deque.appendright(0)
        self.assertEqual([4, 2, 3, 6, 8, 9, 0], self.deque)

    def testAppendLeft(self):
        self.deque.appendleft(0)
        self.assertEqual([0, 4, 2, 3, 6, 8, 9], self.deque)

    def testRotateRight(self):
        self.deque.rotateright(2)
        self.assertEqual([8, 9, 4, 2, 3, 6], self.deque)

    def testRotateLeft(self):
        self.deque.rotateleft(2)
        self.assertEqual([3, 6, 8, 9, 4, 2], self.deque)
        
test = MyDeQueTest()
suite = TestLoader().loadTestsFromModule(test)
TextTestRunner().run(suite)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

Een dergelijke evaluator is onveilig. Als de gebruiker in een Linux-omgeving de expressie `os.system('rm -rf *')` intikt, kan er veel schade worden aangericht. Je kunt dit voorkomen door bij de eval-functie een dictionary mee te geven.

Schrijf de klasse SimpleSecureEvaluator. De klasse bevat een dictionary-attribuut `d` en een methode `eval`.

De klasse wordt gebruikt bij interactieve sessies, waarbij de gebruiker het volgende kan invoeren:
* een toekenningsopdracht. De opdracht van de vorm `<variabele> = <expressie>` wordt opgeslagen in dictionary-attribuut `d`. Hierbij wordt <variabele> als key-waarde opgeslagen. De expressie wordt eerst geëvalueerd en het resultaat wordt als value-waarde bij de key-waarde opgeslagen. Ook opdrachten van de vorm `i = i +1` zijn toegestaan, maar alleen variabelen van voorafgaande toekenningsopdrachten mogen in de expressie voorkomen.
* een expressie. Alleen variabelen van voorafgaande toekenningsopdrachten mogen in de expressie voorkomen. Bij de evaluatie van de expressie wordt gebruik gemaakt van `d`. Het resultaat wordt aan de gebruiker getoond.
* de opdracht `mydir`. De dictionary `d` wordt dan getoond.

In [13]:
import re

class SimpleSecureEvaluator(object):
    def __init__(self, d={}):
        self.d = d

    def eval(self, expression):
        if expression.strip() == "mydir":
            [print("{}: {}".format(k, v)) for (k, v) in self.d.items()]
            return

        if expression.find('=') > -1:
            var, innerExpr = expression.split('=', 1)
            var = var.strip()

            if not var.isalpha():
                raise ValueError(
                    "A variable name should only exists out of alpha characters: {}".format(var))
            self.d[var] = self.eval(innerExpr)
            return

        expressionList = re.sub(
            '(\(|\)|[+\\-*\^\/=])', ' \\1 ', expression).split()

        for item in expressionList:
            if item.isalpha():
                if item in self.d.keys():
                    expression = expression.replace(item, str(self.d[item]))
                else:
                    raise ValueError(
                        "Invalid expression! {} not defined!".format(item))

        return eval(expression)


evaluator = SimpleSecureEvaluator()

evaluator.eval("cv = 1 + 1")
evaluator.eval("cv = cv + 2 + 3")
#evaluator.eval("print('Hello World!')")
evaluator.eval("mydir")


cv: 7


#### Opdracht 2.3.1 — Introspectie van functies
Schrijf een programma dat van een gegeven pyc-file (`function_demo.pyc`) nagaat welke functies de file bevat. Bepaal bij iedere functie de signature en de documentatie. Test de functies.

Ga als volgt te werk:
* Pas de opdracht `dir(<modulename>)` toe. Het resultaat is een lijst van element-namen van de file. De lijst bevat strings.
* Bepaal van ieder element het type. Gebruik hierbij de eval-functie.
* Maak gebruik van het volgende: voor een functie `f` geldt: `type(f).__name__ = 'function'`.
* Gebruikt de functies `signature` en `getdoc` van de module `inspect`.

In [ ]:
# import function_demo_3_5 # Select correct import for your interpreter version
# import function_demo_3_6

# TODO: zie hierboven

#### Opdracht 2.3.2 — Introspectie van klassen
Schrijf een programma dat van de gegeven pyc-file (`class_demo.pyc`) nagaat welke klassen de file bevat.

Ga te werk zoals bij opdracht 2.3.1.

Bedenk daarbij het volgende:
* Voor een klasse `C` geldt: `type(c).__name__ == 'type'`
* Voor een methode `m` geldt: `type(m).__name__ == 'function'`

In [ ]:
# import class_demo_3_5
# import class_demo_3_6

# TODO: zie hierboven

#### Opdracht 2.4 — Decorators [ Portfolio]
Een decorator is een functie die andere functies aanpast om zodanig (aspect-oriented) functionaliteit toe te voegen. Denk bijvoorbeeld aan de optie op een timing toe te  voegen (hoe lang duurt het uitvoeren van een bepaalde functie-aanroep), of security aspecten (alleen als het juiste pincode/password wordt ingevoerd na aanroep zal de functie ook echt worden uitgevoerd).

Het basispatroon van een decorator is het volgende:

Herschrijf de decorator hierboven zodat hij naar het scherm schrijft welke functie (naam) uit welke module (naam) er gedraait is. Je kan deze decorator vervolgens toepassen bij de volgende opdracht, als je kan achterhalen hoe je hem toevoegd aan __alle__ functies van __alle__ modules van de simulator.

In [14]:
from functools import wraps

def myDecorator(f):
    # Definieer een binnenste functie, die je om de 'oude' functie heen 'wrapt'
    @wraps(f) #Gebruik van wraps niet noodzakelijk (zie reader), maar wel netter
    def inner(*args, **kwargs): 
        # gebruik van list arguments en dict keyword arguments omdat we niet weten wat voor functie we gaan krijgen.
        print("Name {}".format(f.__name__))
        print("Module: {}".format(f.__module__))
        result = f(*args, **kwargs)
        # mogelijk nog meer nuttige aanroepen hier...
        return result
    return inner

# Toepassen d.m.v. '@' alleen mogelijk bij definitie van de functie!
@myDecorator
def successor(x):
    return x+1

print(successor(3))
print('-'*20)

# Maar het kan ook door 'overschrijven' van eerder gedefinieerde functie (= hernoemen van de functie-pointer)
from math import sqrt
print(sqrt(4)) # nog niets verandert

print('-'*20)

sqrt = myDecorator(sqrt)
print(sqrt(4)) # nu wel!


Name successor
Module: __main__
4
--------------------
2.0
--------------------
Name sqrt
Module: math
2.0


#### Eindopdracht
Gebruik de beschrijving van de simulator in hoofdstuk 6.1 van de reader en introspectie om:
* Het functioneren van de simulator te achterhalen;
* Tests te kunnen draaien om de correctheid van de simulator te achterhalen (functioneert de simulator vergelijkbaar met de fysieke opstelling?);
* De controller unit van de simulator te vervangen door een eigen geschreven variant.

De Python-byte code van de simulator is toegevoegd in de folder `simulator-3.5` (voor Python 3.5) en `simulator-3.6` (voor Python 3.6). Gebruik `main.py` om de simulator te starten.   
De GUI vereist de installatie van PyGame; gebruik `pip3 install pygame` in een prompt om de pygame-module te installeren (draai als Admin op Windows).